*Human_MIDI_Quantizer.ipynb* <p style='text-align: right;'> <b> September 20th 2020 </b> </p>
<p style='text-align: right;'> <b> David Diston </b> </p>

# Quantize Note Timings for Human Performance Midi Files

***These files will be used when training a model to classify Human Performances and Computer Generated Performances***

This code will sort through each Midi file in the project `Raw_Midi` folder:
1. Each file will be copied and processed to bin all note timings
2. Quantization for Computer Performances results in binning to the nearest 32th note (quantization = 8)
    * All pieces processed are in 4/4 time
    * Therefore Quantization should be equal to 1, or x where x = $2^{y}$ and where y is a whole number
3. Each processed file copy will be saved as a new 'Human Quantized Midi' file in the `Human_Performance_Midi` folder

In [1]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
import os

In [2]:
'''********************************************'''
quantization = 8
'''********************************************'''

# Create a count of all files processed
file_count = 0

# I will iterate over all performance files in Raw_Midi

for file in os.listdir('Raw_Midi/'):

    clip = MidiFile(f'Raw_Midi/{file}')

    # Using the quantization value, I can calculate the time ticks per beat quantization
    ticks_per_quant = int(clip.ticks_per_beat / quantization)
    # Attempting to quantize to a finer scale than the music is encoded with cause an error
    assert clip.ticks_per_beat % quantization == 0, 'ERROR: Quantization if too Fine.'

    # Counting variables
    cum_time = 0
    btw_note = 0

    # This loop will return the message number of the first note_on message
    for note_msgs, msg in enumerate(clip.tracks[0]):
        if msg.type == 'note_on':
            break

    # This variable will identify the note that switched from one quantization bin to the next
    switch = False

    # Here I will iterate over all messages starting with the first note_on message found above 
    for msg in clip.tracks[0][note_msgs:]:

        # All times are time deltas, so I will create a cumulative time variable
        cum_time += msg.time

        # Here I am using the time ticks per quantum and the cumulative time to bin each note
        '''
        If the remainder of cum_time/ticks_per_quant is less than 75% of the distance from one quantization 
        to the next, then the note is binned with a time delta of 0 (back to the previous beat).
        If the remainder is greater than 75% and switch is false, then the message time delta will be set
        to the number of ticks per quant (bin size), and switch will be set to true (binned to the next beat).
        The switch variable will ensure that only the first note of all notes falling between > 75% of
        a quantum, and the next quantum will be coded with the new quantum time.
        Once the new quantum time delta has been established, all other note_on messages will be binned
        with a time delta of zero.
        '''
        if (cum_time % ticks_per_quant) <= (ticks_per_quant * 0.75):
            msg.time = 0
            switch = False

        elif (cum_time % ticks_per_quant) > (ticks_per_quant * 0.75) and switch == False:
            msg.time = int(ticks_per_quant)
            switch = True

        elif (msg.type == 'note_on'):
            msg.time = 0

    # Since time deltas have been changed as a function of the quantization, the time must be ajusted
    # to maintain the original tempo
    for msg in clip.tracks[0][note_msgs:]:
        msg.time = int(msg.time * (quantization / 4))

    # Before saving, I want to make sure that all times less than the quantization time are set to an
    # integer value of zero
    # This will also bin all note_off messages back to the nearest bin (32nd note)
    for msg in clip.tracks[0][note_msgs:]:
        if msg.time < int((ticks_per_quant) * (quantization / 4) - 1):
            msg.time = int(0)

    # I will save the new quantized midi file
    clip.save(filename = f'HumComp/HumComp_QUANT/Human/{file}')
    
    file_count += 1    
    print(f'Processed {file_count} files.', end = '\r')
    
print('\nDone')

Processed 2583 files.
Done


<p style='text-align: right;'> <b> Next Step: </b> Quantize Computer Performance Midi Files - <em> Computer_MIDI_Quantizer.ipynb </em> </p>